In [1]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'
file_names = ['Serie_A']


In [2]:
years = range(2013,2024)


for year in years:


    print(year)
    df_jogos_2014 = pd.read_json(f'../Data/Serie_A_{year}_games.json')
    df_jogos_2014_ = df_jogos_2014.T


    for index, row in df_jogos_2014_.iterrows():
        if index-1 < len(df_jogos_2014_):
            processor = FootballDataProcessor(df_jogos_2014_, index-1)
            final_df = processor.process(data_folder, file_names, year)
            final_df.rename(columns={'player_id': 'player'}, inplace=True)

            final_df['player'] = final_df['player'].astype('string')
            id_players['player'] = id_players['player'].astype('string')

            final_df_ = pd.merge(final_df, id_players, on='player', how='left')

            # Verificar se o índice está dentro dos limites

            team_1_name = df_jogos_2014_.iloc[index-1, 0]
            team_2_name = df_jogos_2014_.iloc[index-1, 1]

            # Criar uma instância de FootballMatchRating
            rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

            # Atualizar as ratings
            updated_match_data = rating_calculator.update_ratings()
            # Garantir que 'player' em updated_match_data está como string
            updated_match_data['player'] = updated_match_data['player'].astype('string')
            # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
            updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
            updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

            updated_values = id_players[['player']].merge(
                updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
                on='player',
                how='left'
            )

            # Atualizar apenas as linhas corespondentes em id_players
            id_players.update(updated_values)
        else:
            print(f"Índice {index-1} fora dos limites para o ano {year}")
            continue




id_players


2013
2014
2015
2016
Índice 379 fora dos limites para o ano 2016
2017
2018
2019
2020
2021
2022
2023


,player,rating,age,games_played,player_name,time_jogador
0,359696,1406.244108,25,197,16Deyverson Deyverson Brum Silva,Cuiabá / MT
1,613263,1465.883019,25,53,72Pedro Morisco Pedro Luccas Morisco ...,Coritiba / PR
2,522895,1482.279234,25,64,9Braian Rod ... Braian Damian Rodrig ...,Grêmio / RS
3,304285,1498.643726,25,54,21Jonh Lennon Jonh Lennon Silva Santos,Cruzeiro / MG
4,749291,1507.408018,25,51,28Pablo Garcia Pablo Javier Garcia ...,Coritiba / PR
...,...,...,...,...,...,...
3082,392189,1489.940452,25,51,11Dalberson Dalberson Ferreira d ...,Cruzeiro / MG
3083,543163,1587.214030,25,107,25Zanocelo Vinicius Nelson de S ...,Fortaleza / CE
3084,567402,1439.916929,25,62,65Rodrigo Va ... Rodrigo Santos Varanda,América / MG
3085,181106,1453.390839,25,89,33Guilherme Guilherme Oliveira Santos,Juventude / RS


In [3]:
x = id_players.sort_values(by='rating', ascending=False)
x.to_excel('teste.xlsx')
x

,player,rating,age,games_played,player_name,time_jogador
353,176390,1752.039428,25,298,2Marcos Rocha Marcos Luis Rocha Aquino,Palmeiras / SP
2548,169050,1711.520492,25,391,21Weverton Weverton Pereira da Silva,Palmeiras / SP
2262,392224,1704.316769,25,141,26Murilo Murilo Cerqueira Paim,Palmeiras / SP
902,389378,1691.684907,25,143,19Breno Breno Henrique Vasco ...,Palmeiras / SP
1707,541232,1691.481893,25,126,8Cuellar Gustavo Leonardo Cue ...,Flamengo / RJ
...,...,...,...,...,...,...
2045,173760,1330.941530,25,156,6Carlinhos Carlos Andrade Souza,CSA / AL
187,593809,1329.590105,25,112,16Jonatan Gomez Jonatan David Gomez,Sport / PE
574,143465,1319.212913,25,170,99Alecsandro Alecsandro Barbosa F ...,CSA / AL
2572,320196,1304.403499,25,88,12Jordi Jordi Martins Almeida,CSA / AL
